In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.saving import load_model
import numpy as np
from Dataset import load_dataset, create_set

In [ ]:
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(144, 256, 3))

#resnet.summary()      # (None, 5, 8, 2048) last layer's output shape

# adding custom layers
custom_layers = resnet.output
custom_layers = GlobalAveragePooling2D() (custom_layers)
custom_layers = Flatten() (custom_layers)
custom_layers = Dense(256, activation='relu') (custom_layers)
custom_layers = Dense(8, activation='softmax') (custom_layers)

network = Model(inputs=resnet.input, outputs=custom_layers)

for layer in resnet.layers:
    layer.trainable = False

network.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# cheking if everything is correct
#network.summary(show_trainable=True)

In [ ]:
dataset = load_dataset('./../dataset/new_dataset.csv', './../dataset/images/')

In [ ]:
classes = [4] + [9+i for i in range(7)]
x_train, y_train, remaining = create_set(dataset, classes, 500)

print(x_train.shape, y_train.shape)

In [ ]:
x_val, y_val, remaining2 = create_set(remaining, classes, 66)

print(x_val.shape, y_val.shape)

In [ ]:
x_test, y_test, _ = create_set(remaining2, classes, 66)

print(x_test.shape, y_test.shape)

In [ ]:
network.fit(x=x_train, y=y_train, 
            batch_size=16, epochs=15, 
            validation_data=(x_val, y_val))

In [ ]:
count = 0
for x, y in zip(x_test, y_test):
    prediction = network.predict(x.reshape(1, 144, 256, 3))

    print(f"pred: {np.argmax(prediction)}, \ttrue: {np.argmax(y)}")
    
    if np.argmax(prediction) == np.argmax(y):
        count += 1

print(f'{count}/{len(x_test)} images classified correctly.')

In [ ]:
# unfreeze last layers
for layer in network.layers[-24:]:
    layer.trainable = True

# really small learning rate
network.compile(optimizer=Adam(0.000001), loss='categorical_crossentropy', metrics=['accuracy'])

# cheking if everything is correct
#network.summary(show_trainable=True)

In [ ]:
network.fit(x=x_train, y=y_train, 
            batch_size=16, epochs=20, 
            validation_data=(x_val, y_val))

In [ ]:
network.save('finetune_2.keras')

In [ ]:
network = load_model('finetune_2.keras')